# Wave with periodic bc with the scheme from Palma

We solve the wave equation with Dirichlet boundary conditions (homogeneous) up to $T = 2L$ where $L$ is the space size of the integrating region. The data is reflection symmetric, $u_0(x) = u_0(L-x)$, $\partial_t u(x,t=0) = 0$. This way at the last point the solution repeats itself and we can measure the error very well.

In [ ]:
import Pkg
Pkg.add("Zygote")
#Pkg.instantiate()
#Pkg.add("StatsBase")
using ComponentArrays
using Distributions
#using WGLMakie
using Lux
using LuxCUDA
using OptimizationOptimJL
using Random
using UnPack
using Zygote
#using CairoMakie  # Backend que funciona en notebooks
using StatsBase
using Revise
#import NaNMath

I put some functions here.

In [ ]:
includet("neural_tools.jl")

In [ ]:
# -------------------------------------------------------------------
# Configuración
# -------------------------------------------------------------------
config = Dict(
    :N_input => 2,          # [x; t]
    :N_neurons => 20,
    :N_layers => 3,
    :N_output => 1, 
    :N_points => 15000,     # puntos de colisión (x,t)
    :xmin => 0.0,
    :xmax => 5.0,           # = L dominio espacial
    :tmin => 0.0,           # t_min
    :tmax => 10.0,           # t_max
    :optimizer => BFGS(),
    :maxiters => 3000,
    # for the initial data
    :A => 0.010,
    :x0 => 2.0,
    :x1 => 3.0,
    :p => 2,
    :c => 1.0
)

**Note 1:** I attempted to put a bump function of compact support (with some if's) and could not make it run. It seems there are problems with the AD scheme to handle if's. So I put a simpler function.

**Note 2:** As it is, with adaptive it takes 12 hours (CPU) to run... And the solution is wrong...

In [ ]:
# -------------------------------------------------------------------
# Second order wave equation (onda 1D): u_tt - c^2 u_xx = 0
# -------------------------------------------------------------------
wave_equation(∂2u_∂x2, ∂2u_∂t2, c) = ∂2u_∂t2 .- (c^2) .* ∂2u_∂x2

```
Residual at collocation points
``` 

function residual_at_points_Dirichlet(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    return vec(abs.(res |> cpu_device()))  # magnitud del residuo en CPU
end


# -------------------------------------------------------------------
# Loss function with Dirichlet boundary conditions hard enforced.
# -------------------------------------------------------------------

function loss_function(input, NN, Θ, st)
    res = residual_at_points_Dirichlet(input, NN, Θ, st)
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return log10(sum(abs2, res) / length(res))
end

#=
function loss_function(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return log10(sum(abs2, res) / length(res))
end
=#

# -------------------------------------------------------------------
# Callback
# -------------------------------------------------------------------
function callback(p, l, losses)
    push!(losses, l)
    println("Current loss: ", l)
    return false
end

In [ ]:
loss_function(input, NN, Θ, st)

In [ ]:
# -------------------------------------------------------------------
# Entrenamiento
# -------------------------------------------------------------------
losses = Float64[]

NN, Θ, st = create_neural_network(config)
input = generate_input_x_t(config)

@show typeof(input) size(input)



In [ ]:


#calculate_Dirichlet_f(input[1:1, :], input[2:2, :], NN, Θ, st)
f, ∂2f_∂x2, ∂2f_∂t2 = calculate_derivatives_Dirichlet(input[1:1, :], input[2:2, :], NN, Θ, st)

f

In [ ]:
#adaptive = true
adaptive = false

if adaptive 
    # Configura las rondas adaptativas (ajusta a tu gusto)
    nrounds = 20                # nº de rondas RAD
    iters_per_round = 500       # iteraciones BFGS por ronda
    #k1, k2 = 1.0, 1          # hiperparámetros RAD
    Ntest = 200_000              # candidatos por ronda

    for r in 1:nrounds
        @info "RAD round $r / $nrounds  |  iters=$iters_per_round"
        # Optimiza sobre el conjunto actual de colisión
        optf   = OptimizationFunction((Θ, input) -> loss_function(input, NN, Θ, st), AutoZygote())
        optprob = OptimizationProblem(optf, Θ, input)
        optres  = solve(
            optprob,
            config[:optimizer];
            callback = (p, l) -> callback(p, l, losses),
            maxiters = iters_per_round,
        )
        Θ = optres.u  # continúa desde el óptimo de la ronda

        # Re-muestrea puntos de colisión ponderando por residuo
        input = adaptive_rad(NN, Θ, st, config; Ntest=Ntest, Nint=config[:N_points])#, k1=k1, k2=k2)
    end

else
    @info "Training with $(config[:N_points]) collocation points and $(config[:maxiters]) iterations"

    optf = OptimizationFunction((Θ, input) -> loss_function(input, NN, Θ, st), AutoZygote())
    optprob = OptimizationProblem(optf, Θ, input)

    optresult = solve(
        optprob,
        callback = (p, l) -> callback(p, l, losses),
        config[:optimizer],
        maxiters = config[:maxiters],
    )
end

# Parámetros optimizados a CPU si procede
Θ = optresult.u |> cpu_device()


In [ ]:
ax = (title = "loss vs iterations")
fig, ax = lines(losses, label = "Loss")
axislegend()
fig

In [ ]:
@save "wave_Dir_1.jld2" config domains eqs bcs depvars θ st loss

In [ ]:
ax = (title = "solution")
t = 10.0
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

#xs[1,1]
fig, ax = lines(xs[1,:],sol_p, label = "t=0")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
lines!(xs[1,:],-S*config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4, label = "initial condition", linestyle = :dash)     
axislegend()
fig

In [ ]:
initad = [-config[:A]*(xs[1,i] .- config[:xmin]).^4 .* (xs[1,i] .- config[:xmax]).^4 for i in 1:length(xs[1,:])]
S = maximum(-sol_p)/maximum(-initad)

In [ ]:
ax = (title = "solution")
t = 0.0
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

fig, ax = lines(xs[1,:],sol_p .+ S*(config[:A]*(xs[1,:] .+ config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4), label = "t=0")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
#lines!(xs[1,:],sol_p .-config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4, label = "initial condition", linestyle = :dash)     
axislegend()
fig

**Save data:**

In [ ]:
#@save "wave_dirichlet_1D_adaptive.jld2" NN Θ st config losses
@save "wave_dirichlet_1D.jld2" NN Θ st config losses